# Create data base for NN 

In [14]:
!pip install pymongo
import pymongo
import json
from bson import json_util

**credentials**

In [15]:
dbname = "lol2"
user = 'lol2i'
passw = 'GoogleTest19'
ip = "5.196.92.209"
port = 35420

#### connection string

In [16]:
client = pymongo.MongoClient(ip,port,username=user,password=passw,authSource=dbname)

In [17]:
db = client[dbname]

**How many games**

In [18]:
db.games.count()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


69395

In [19]:
def downloadCollection(db,collection_name,file_out):
    
    op = open(file_out,"w")
    coll = db[collection_name]
    results = coll.find({})
    i = 0
    for res in results:
        jdump = json.dumps(res,default=json_util.default)
        op.write(jdump+"\n")
        if(i%1000 == 0):
            print(i)
        i += 1
    op.close()

In [20]:
def getResultsByParticipant(gID):
    pgameid=db.games.find({'gameId' :gID})
    for i, g in enumerate(pgameid):
        participants=g['participants']
        result=[]
        for i, participant in enumerate(participants):       
            result.append((i, participant['stats']['win']))
        return result


In [21]:
def getPosition(timelines_frames, player):
    events_and_pos=[]
    for frame in timelines_frames:
        if('position' in frame['participantFrames'][player]):
            position = frame['participantFrames'][player]['position']
            typeList=[]
            events=frame['events']           
            for event in events:
                type_= event['type']
                if 'creatorId' in event:
                    if (event['creatorId']==int(player)):
                        typeList.append((type_))
                elif 'participantId' in event:
                    if (event['participantId']==int(player)):
                        typeList.append((type_))
                elif 'killerId' in event:
                    if (event['killerId']==int(player)):
                        typeList.append((type_))    
            x = position['x']
            y = position['y']
            tl=typeList
            
            try:
                events_and_pos.append((x,y,tl))
            except Exception as e:
                pass
            
    return events_and_pos

In [22]:
time_line_all=db.timelines.find({})

In [23]:
# save object to a file
# HIGHEST_PROTOCOL for binary format
# 0 for text output
import pickle
def save_obj(obj, name ):
    with open('DataNN/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, 0)

In [24]:
X=[]
Y=[]


for i, gameId in enumerate(time_line_all):
    
    # for every game
    gID= gameId['gameId']
    try:
        resGame= getResultsByParticipant(gID)

        if 'timeline' in gameId:
            tFrames=gameId['timeline']['frames']
            for j in range(0,10):     # player 1 to 10
                try:
                    events_and_pos=getPosition(tFrames, str(j+1))
                    X.append(events_and_pos)
                    for r in resGame:
                        if r[0]==j:
                            Y.append(r[1])
                except Exception as e:
                    pass
            if i==20000:
                save_obj(X, "X_seq20000") #max workable in notebook with this machine power
                save_obj(Y, "Y_seq20000")
                break
            if i%1000==0:
                print('--->', i)
            
    except Exception as e:
        pass

---> 0
---> 1000
---> 2000
---> 4000
---> 5000
---> 6000
---> 7000
---> 8000
---> 9000
---> 10000
---> 11000
---> 12000
---> 13000
---> 14000
---> 15000
---> 16000
---> 17000
---> 18000
---> 19000
